# Segunda Entrega: Pronóstico de Demanda Hotelera en Mendoza
Este notebook documenta el proceso de generación de predicciones de demanda hotelera en Mendoza utilizando el mejor modelo seleccionado.

## 1. Cargar el registro del modelo
En esta sección se lee el archivo JSON de registro para obtener información del mejor modelo y sus características.

In [ ]:
import json
from pathlib import Path

# Ruta al archivo de registro del modelo
registry_path = Path('data/models/registry.json')

with open(registry_path, 'r') as f:
    registry = json.load(f)

print('Modelo seleccionado:', registry['name'])
print('Características:', registry)

## 2. Cargar el dataset histórico
Se carga el archivo Parquet con los datos históricos curados y se ordenan por fecha.

In [ ]:
import pandas as pd
from pathlib import Path

# Ruta al archivo Parquet curado
curated_path = Path('data/curated/mza_turismo_curated.parquet')

df = pd.read_parquet(curated_path)
df['fecha'] = pd.to_datetime(df['fecha'])
df = df.sort_values('fecha')
df.head()

## 3. Construir fechas futuras y features
Generar un DataFrame con fechas futuras y calcular las variables necesarias para el modelo (mes, año, sin/cos del mes).

In [ ]:
import numpy as np

# Definir horizonte de predicción
horizon = 12  # meses
last_date = df['fecha'].max()
future_idx = pd.date_range(last_date, periods=horizon + 1, freq='M')[1:]
fut = pd.DataFrame({'fecha': future_idx})
fut['mes'] = fut['fecha'].dt.month
fut['anio'] = fut['fecha'].dt.year
fut['sin_m'] = np.sin(2 * np.pi * fut['mes'] / 12)
fut['cos_m'] = np.cos(2 * np.pi * fut['mes'] / 12)
fut.head()

## 4. Generar predicciones con el modelo
Cargar el modelo seleccionado, realizar predicciones sobre las fechas futuras y anexar metadatos relevantes.

In [ ]:
import joblib

# Cargar el modelo
model_path = Path(registry['path'])
model = joblib.load(model_path)
feats = registry['feats']

# Generar predicciones
preds = model.predict(fut[feats])
fut['prediccion'] = preds
fut['modelo'] = registry['name']
fut['mae'] = registry.get('mae', None)
fut.head()

## 5. Guardar las predicciones en Parquet
Persistir el DataFrame de predicciones en formato Parquet, particionado por fecha de ejecución.

In [ ]:
from datetime import datetime

# Definir carpeta de salida y nombre de archivo
output_folder = Path('data/curated')
output_folder.mkdir(parents=True, exist_ok=True)
fecha_ejecucion = datetime.now().strftime('%Y%m%d')
output_path = output_folder / f'mza_turismo_forecast_{fecha_ejecucion}.parquet'
fut.to_parquet(output_path, index=False)
print('Predicciones guardadas en:', output_path)